# NiftiファイルからZ-scoreを自動抽出  
---  
  
引数：4D化した.niiファイルがあるフォルダまでのパス  
  
---  
  
入力：4D化した.niiファイル，[部位名]mask.niiファイル（chunks_list.csv, targets_list.csv）  
   
---
  
出力：（chunks_list.csv, targets_list.csv）  
  
---  
  
注意：このプログラムはUbuntuでやったほうがいいかも（抽出する部位が多いほどデータが重くなってメモリがたりなくなる！）  
  

注意：このプログラムを実行する前に以下の作業をすること  

3D脳画像データ（前処理済みの.niiファイル）を4Dに変換  
  
1. SPM起動
2. Batchを開く
3. 画面左上SPM → Util → 3D to 4D File Conversion
3. 3D Volumesから4Dに変換したいファイル('swr'のついたniftiファイル全て)を選択
4. Output Filenameでファイル名を指定
5. 再生ボタンを押す

Motor mask（Z-score化したいボクセルの位置）の作成  
※ wfu_pickatlasはmatlabサーバーで実行（Macだと.nii形式で保存できなかった） 
※ 中山メモ：ubuntuでshareで作業するにはmatlabまで移動してshareを開く，基本的にsudoを使って作業（matlabもsudoで起動）    
 
1. wfu_pickatlasフォルダをダウンロード（ネットから or Shareかnakayamaフォルダのどっかにある）
2. MATLABでwfu_pickatlasフォルダのパスを通す
3. MATLABコマンドライン上で > wfu_pickatlas で起動
4. 画面左からマスクしたい部分を選択（全脳の場合は TD Hemispheres を全選択）
5. SAVE MASKからマスク画像をmaskとして保存（名前に'-'とか入れない）
6. SPM → fMRI起動
7. MenuからNormalize(Est & Wri)を選択
8. Bath Editorが立ち上がったらDataをダブルクリック
9. Image to Alignには'swr'がファイル名についているniftiファイルを適当に1つ選択
10. Image to Writeには先ほど作成したmaskファイルを選択
11. 再生ボタンを押す → wmask.niiができる
12. Batch Editorに戻りRealign(Reslice)を選択
13. Imagesをダブルクリック
14. 'swr'がファイル名についているniftiファイルを適当に一つ，先ほどできたwmask.niiファイルの順で選択
15. 再生ボタンを押す → rwmask.niiができる  
  
Active masの作成
1. MATLABコマンドライン上で > wfu_pickatlas
2. ANALYSISのボタン押す
3. SPM.matを選択する
4. SPMでResultを出す要領でp値などを設定，基本的にnoクリック
5. Moror maskの5以降と同様
  
※作成したrwmask.niiはどの部位のマスクかわかるように名前を変えている  
・maskAll.nii：全脳  
・rwmask12346.nii：ブロードマンエリア1,2,3,4,6

In [1]:
from mvpa2.suite import *
from mvpa2.datasets.mri import fmri_dataset
import os
import os.path
from os.path import join as pathjoin
from pprint import pprint
# from nifti import NiftiImage
import glob
import numpy as np
import pandas as pd
import sys
import pickle
# import dill
import csv

import nibabel as nib

/home/nakayama/.pyenv/versions/anaconda3-5.3.0/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized
Failed to import duecredit due to No module named 'duecredit'
/home/nakayama/.pyenv/versions/anaconda3-5.3.0/lib/python3.7/site-packages/h5py/_hl/base.py:19: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import (Mapping, MutableMapping, KeysView,
/home/nakayama/.pyenv/versions/anaconda3-5.3.0/lib/python3.7/site-packages/nibabel/testing/__init__.py:21: DeprecationWarning: Importing from numpy.testing.decorators is deprecated, import from numpy.testing instead.
  from numpy.testing.decorators import skipif
/home/nakayama/.pyenv/versions/anaconda3-5.3.0/lib/python3.7/site-packages/nibabel/extern

コマンドライン引数で.niiファイルがあるディレクトリまでのパスを取得．

In [42]:
#args = sys.argv
#PATH_pre = args[1]

# jupyter notebookのときはここで指定
PATH = '../../Data_mri/tappingState-2fe/20181029rn/64ch/'

# 総スキャン数
scan_num = 192

# 前処理済みならswrがついた.niiファイルを選択
PATH_nii = PATH + '4D.nii'
PATH_mask = PATH + 'rwmaskAll.nii'


In [43]:
headcoil = PATH.split('/')[5]

if headcoil == 'mb':

    # 1タスクのスキャン数
    taskNum = 296
    
elif headcoil == '64ch':

    # 1タスクのスキャン数
    taskNum = 96



## splitRT関数

引数に１ブロックのscan数を受け取り，Rest時とTapping時のデータを分けてcsvファイルで書き出し

In [44]:
def splitRT(brain):

    # Rest，Tappingのデータ抽出用maskを作成
    # （行数 // number_scan）% 2　で割り切れる（0）ならRest，割り切れない（1）のであればTapping
    # （除算は '//' としないと小数まで計算される）

    maskRest = ([True] * taskNum) + ([False] * taskNum)
    maskTapping = ([False] * taskNum) + ([True] * taskNum)
    
    # mask適用
    dataRest = brain[maskRest]
    dataTapping = brain[maskTapping]
    
    
    # csv書き出し
    PATH_rest = PATH + 'raw_rest.csv'
    dataRest.to_csv(PATH_rest, index = False)
    
    PATH_tapping = PATH + 'raw_tapping.csv'
    dataTapping.to_csv(PATH_tapping, index = False)

# main関数

* fMRIデータ（niftiファイル）読み込み
* 全ボクセルデータZ-score化，抽出，書きだし

In [45]:
if __name__ == '__main__':

    # 4D化した.niiファイル名リストを作成

    nifti = [PATH_nii]


In [46]:
    # 教師データの作成（この作業なしにやる方法がわからないので，必要のない作業ではあるがやる）

    task = ['0'] * int(scan_num / 2)
    task2 = ['1'] * int(scan_num / 2)

    task.extend(task2)
    target = pd.DataFrame(task)

    PATH_target = PATH + 'targets_list.csv'
    target.to_csv(PATH_target, index = False, header = None)

    targets_list = []

    targets_file = open(PATH_target, 'rU')
    dataReader = csv.reader(targets_file)

    for row in dataReader:
        targets_list.append(row[0])

/home/nakayama/.pyenv/versions/anaconda3-5.3.0/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: 'U' mode is deprecated
  


In [47]:
    # チャンク（試行数リスト？）の作成（この作業なしにやる方法がわからないので，必要のない作業ではあるがやる）

    chunk = ['1'] * scan_num

    chunks = pd.DataFrame(chunk)

    PATH_chunk = PATH + 'chunks_list.csv'
    chunks.to_csv(PATH_chunk, index = False, header = None)

    chunks_list = []

    chunks = open(PATH_chunk, 'rU')

    for x in chunks:
        chunks_list.append(x.rstrip('\r\n'))

    chunks.close()


/home/nakayama/.pyenv/versions/anaconda3-5.3.0/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: 'U' mode is deprecated
  if sys.path[0] == '':


In [48]:
    # データセットの整形

    dataset = fmri_dataset(nifti, targets=targets_list, chunks=chunks_list, mask=PATH_mask ,sprefix='voxel', tprefix='time', add_fa=None)

    poly_detrend(dataset, polyord=1, chunks_attr='chunks')

    dataset = dataset[np.array([l in ['0', '1']
                               for l in dataset.targets], dtype='bool')]

/home/nakayama/.pyenv/versions/anaconda3-5.3.0/lib/python3.7/site-packages/mvpa2/mappers/detrend.py:325: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  fit = np.linalg.lstsq(regs, ds.samples)


In [49]:
    # Z-score抽出 → データフレーム化
    zscore(dataset, chunks_attr='chunks', param_est=('targets', ['0']), dtype='float32')

    VoxelData = pd.DataFrame(np.array(dataset[:,:]))
    print(VoxelData .shape)

(192, 70455)


In [50]:
    voxName = []
    
    for i in range(len(VoxelData.columns)):
        
        name = 'Voxel' + str(i+1)
        
        voxName.append(name)
    
    VoxelData.columns = voxName

In [51]:
    # 全データのcevファイル書き出し
    PATH_all = PATH + 'all_raw.csv'
    VoxelData.to_csv(PATH_all, index = False)

In [52]:
    splitRT(VoxelData)

In [19]:
# もしデータセットの整形でサイズエラーが出た時は以下で.niiファイル読み込みしてサイズを確認する
nii0 = nib.load('../../Data_mri/tappingState-2fe/20181029rn/64ch/wmaskAll.nii')
img0 = nii0.get_data()
img0.shape

(79, 95, 79)